In [ ]:
import holoviews as hv
import numpy as np
import xarray as xr
hv.extension('bokeh', logo=False) 

In [ ]:
dx = 5
nxmax, nymax = 200, 20
zb = np.zeros((nxmax, nymax))

ib = 0.001
Hs = 1.0
for i in range(nxmax) : zb[i,:] = Hs - ib * i * dx

zb[:60, :5]  += 5.0 
zb[:60, 15:] += 5.0 

In [ ]:
def mk(ind):
    dp = ds.isel(t=ind)
    U = dp['u'].values.T
    V = dp['v'].values.T
    mag = np.sqrt(U**2 + V**2)
    angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)
    
    zbn = dp['dzb'] + zb
    wl = zbn + dp['depth']
    
    img = hv.Image((dp['x'],dp['y'],dp['dzb'].values.T), kdims=['x','y']).options(height=220, width=900,cmap='bwr',colorbar=True).redim(z={'range':(-1,1)})
    vec =  hv.VectorField((dp['x'], dp['y'], angle, mag))
    
    cont = img * vec * hv.HLine(y=50) * hv.VLine(x=500)
    
    line1 = hv.Curve( (wl[100,:] , dp['y']), kdims='elevation',vdims='y'  ).options(height=220, color='blue')  \
            * hv.Curve( (zbn[100,:], dp['y']), kdims='elevation',vdims='y'  ).options(color='black', title='x=500') \
            * hv.Curve( (zb[100,:], dp['y']), kdims='elevation',vdims='y'  ).options(color='red') \
            
    line2 = hv.Curve( (dp['x'], wl[:,10]) , kdims='x',vdims='elevation', label='water level').options(width=900, height=220, color='blue')  \
            * hv.Curve( (dp['x'], zbn[:,10]), kdims='x',vdims='elevation', label='bed').options(color='black', title='y=50') \
            * hv.Curve( (dp['x'], zb[:,10]), kdims='elevation',vdims='y', label='initial').options(color='red')

    return cont, line1, line2 

ds = xr.open_dataset('output_r.nc')

holo1 = hv.HoloMap(kdims=['time'])
holo2 = hv.HoloMap(kdims=['time'])
holo3 = hv.HoloMap(kdims=['time'])

for i in range(0,101,20):
    holo1[i*5], holo2[i*5], holo3[i*5] = mk(i)
    
holo = hv.Layout([holo1, holo2, holo3]).cols(2)
hv.save(holo,'out.html')

ds.close()
